# Low-Order N-Gram Based English Phonotactics Analysis

This project seeks to research:
1. Are there patterns in how English words are constructed? If so, can the patterns be used to replicate the acceptability judgements of a native English Speaker?
2. Are there certain combinations of characters that give a word or a name certain impression? (i.e., are there patterns in female names that gives them a sense of femininity?) 
3. Can low-order n-gram generative models capture English phonotactics, simply by the surface analysis of the characters instead of their underlying sounds? If so, how low can n go until the model fails?

## Challenge Goals

Messy data:
I would have to turn lists of words into a DataFrame. I would have to analyze it with n-gram models and break down each word to an array of characters to do so, and result in many arrays of floats. I would also like other datasets of male and female names on the side and merge them with the word list to replicate actual English phonotactic behaviors. 

New library:
I would like to explore more plots and even interactive ones. I currently have my eyes on altair. My only current ideas are heatmaps, and plots where if you put your cursor over, more information would show up. I think it is important to show the findings intuitively.

## Collaboration and Conduct

Students are expected to follow Washington state law on the [Student Conduct Code for the University of Washington](https://www.washington.edu/admin/rules/policies/WAC/478-121TOC.html). In this course, students must:

- Indicate on your submission any assistance received, including materials distributed in this course.
- Not receive, generate, or otherwise acquire any substantial portion or walkthrough to an assessment.
- Not aid, assist, attempt, or tolerate prohibited academic conduct in others.

Update the following code cell to include your name and list your sources. If you used any kind of computer technology to help prepare your assessment submission, include the queries and/or prompts. Submitted work that is not consistent with sources may be subject to the student conduct process.

In [1]:
your_name = "Gary Zhou"
sources = [
    "Price, E. (November 18, 2007) “MIT's 10000-word list”. Massachusetts Institute of Technology. Retrieved May 14, 2025 from https://www.mit.edu/~ecprice/wordlist.10000",
    "Robin Elise Weiss, Alex Vance. (May 9, 2025) “Top 1,000 Baby Boy Names in the U.S”. for 2025. Social Security Administration. Retrieved June 1, 2025 from https://www.parents.com/top-1000-baby-boy-names-2757618",
    "Robin Elise Weiss, Alex Vance. (May 9, 2025) “Top 1,000 Baby Girl Names in the U.S”. for 2025. Social Security Administration. Retrieved June 1, 2025 from https://www.parents.com/top-1000-baby-girl-names-2757832",
    "Daniel, J., & Martin, J. (2024). N-gram Language Models. https://web.stanford.edu/~jurafsky/slp3/3.pdf",
    "Daniel, J., & Martin, J. (2024). Vector Semantics and Embeddings. https://web.stanford.edu/~jurafsky/slp3/6.pdf",
    "Mathematical equation for perplexity. Adapted from “Perplexity for LLM Evaluation”. Abby Morgan. Retrieved From https://www.comet.com/site/blog/perplexity-for-llm-evaluation/", 
    "ChatGPT, on how python classes can also have the effect multiple different constructors, using unspecified arg and isinstance. (ended up unused)",
    "Altair documentations on data manipulation. https://altair-viz.github.io/user_guide/data.html",
    "ChatGPT's response, given the input 'given an altair bar plot, how can I access data from it? etc, if I want to get the value of the first bar, etc.'",
    "ChatGPT's response, given the input 'how can i bar plot the row vector of a table with altair?'",
    "ChatGPT's response, given the input 'how do i throw exceptions in python?'",
    "Pandas melt method documentation, https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html"
    "Altair documentations on comparative charts. https://altair-viz.github.io/user_guide/compound_charts.html"
]

assert your_name != "", "your_name cannot be empty"
assert ... not in sources, "sources should not include the placeholder ellipsis"
assert len(sources) >= 6, "must include at least 6 sources, inclusive of lectures and sections"

## Data Setting and Methods

Note: Most of the code and processing was done in Java. The results of the processing is what will be analyzed here. This was permitted by the staff (https://edstem.org/us/courses/76716/discussion/6760495). Code is in EMA.java. Java code that performed each data transformation will have their snippets posted below each step.

In this report, I will use "Context" and "Content" to describe the inputs and outputs of an n-gram model.

Data Setting: 10k words compiled by the MIT, and 2000 boy and girl names from the SSA. They all have weird outliers, but there are also enough good cases to ignore them. The model is made to handle lowercase characters only, so some uppercase characters would have to be converted before processing them.

Data Transformation: 

1. Take in a document of words (.txt)
	(It can be tested by trying to print every word in the file and see if the file is being loaded or not.)

Scanner sc = new Scanner(new File(fileName)); // Uses a scanner to read the file.

2. Remove instances of dashes, turn uppercase characters into lowercase for easier processing. <>
	(It can be tested by printing every word in the file and see if it only contains lowercase characters)

String word = "" + nul + stx + sc.nextLine().trim().toLowerCase() + end; 

nul is the placeholder for the trigrams that needs two previous characters to generate the next character; stx (start of text) is the placeholder for bigrams and trigrams that needs one or two previous characters to generate the next character; nextLine is because each word is on it's own line; trim is to remove unnecessary spaces; toLowerCase because the model can't handle anything outside of a-z; end is a special character used to see how many times the end of a word is what follows a letter (represented as etx on the tables).

3. Have an array that keeps track of how many times each of the 26 letters appear in the list of words (unigram, index of the array is equal to the character it represents, 0=a, 1=b…), and a dictionary that keeps track of how many times each 26 letters (or the end of a word) appear after each of the 26 letters (or start of a word) (bigram). The bigram would end up with 27 keys, each with an array with 27 slots for numbers. Higher-order n-grams would be constructed in similar fashion, with dictionary that takes tuples instead of single keys.
	(The unigram array can be tested by searching instances of character with ctrl+f, and see if the total instance of a certain character is matching with the data in the array. The bigram array is harder to test, but intuitions of English can help: c is not likely to be followed by a z, so if there are a lot of cases of c being followed by a z, then something is wrong with how data is stored.)

Example with trigram training: (model.trigram is a Map<String, Vector>)

String s = "" + c + word.charAt(i + 1); // two letter combination that acts as the context for what is generated next.

if (!model.trigram.containsKey(s)) { // if there isn't already an entry for s, add a new one. New entries are dynamically created instead of being fully initialized at the start (like the bigram models), due to there being a lot more possible entries.

    model.trigram.put(s, new Vector(vdim)); // put in a new vector of vdim, which is 27 (26 letters, plus etx)
    
}

    model.trigram.get(s).add(word.charAt(i + 2)); // add this instance of context and content pair into the model.

4. Methods of smoothing are applied (optional):

    Add k smoothing: each number in each array for unigram and bigram models are added by a constant k (usually 0 < k < 1), to make it so that rare cases that are not covered by the dataset will not show up as zero.
	(Simply print the arrays before and after the method, and see if there is an extra k added to each number in the arrays.)

Example with trigram again:

trigram.forEach((key, value) -> { // for each entry in the map...

    trigram.get(key).add(smoothie, 1); // add smoothie, which is a vector of vdims, populated with k, a parameter decided in code. The bigger k is, the less relied on training the model is.
    
});

    Stupid backoff (only for bigrams and beyond): if there are no instances of a letter followed by another (for example, x followed by a), the algorithm goes back one level and tries to find the possibility of the second character with one less precedent for it (following the example, the possibility of a appearing alone), and it keep tries to go back until a none zero value is found. That value is multiplied with a constant (experimentally 0.4) and is plugged back into the original array in its slot.
	(Check the case of x coming before a. The chance might be low enough for backoff to happen, and since a is quite common on its own, one would expect to see the first value in the array when x is input into the dictionary for bigram is quite high, and approximately a portion of the chance of a appearing on its own)

This is really only used in trigrams during generation and perplexity calculation:

double sum = Math.log(model.trigram.getOrDefault(s, model.bigram.get(s.charAt(1))).get(word.charAt(0) - 'a')); // Due to being dynamically created, there can be cases where trigrams can't "follow up" after a character. This is when it falls back to the bigram version with getOrDefault, which turns to the default if normal get returns a null.

5. Normalization: the times that letter x appears after letter y is divided by all cases of x appearing, as an example for a bigram model. This makes it so that the sum of possibility of every number in every array is 1, for any n in n-gram. Now, they can be seen as possibilities.
	(Sum up every index of the arrays and see if it adds up to one. A specific function can be written for that.)

Trigram example again:

trigram.forEach((key, value) -> { // for each entry in the map...

    trigram.get(key).normalize(value.total(false)); // call the normalize function on each vector, which just divides every value by a value, which is the total of the vector values here. This makes it so that the total value of every double stored to add up to one.
    
});

(You might be curious about the false parameter. That is there to make the etx value to not interfere when generating words. Experimentally it works well. This does mean that etx will NOT be normalized along with the rest of the context-content pair.)

6. With more formatting and iterating through the dictionary, we now have a DataFrame for our data. It can now be used to generate words (going through the dictionary/array based on the model used, and use the probability of the next character to determine it, then use that as basis to find the next most likely character, etc.), used for perplexity calculations (or to see how “English” a word is, by seeing the possibility of it being generated by our model), and have plots made of it with altair (heatmaps, etc.).
	(Show the results to native English speakers and see if there are anything surprising for them. Sadly, the name generation function won't be able to be showcased here.)

This is how a bigram model (Map<Character, Vector>) is turned into a csv file (assuming correct imports):

String bigram_name = path + "csv_" + model.name + "_bigram.csv"; // the right file name

File bigram = new File(bigram_name); // making a file with the right file name

try { // has to be used to prevent compiler complaining

    bigram.createNewFile(); // Makes a new file in the indicated path
    
    FileWriter writer = new FileWriter(bigram_name); // Making a thing that can write into files
    
    writer.write(idxToChar + "\n"); // the header row
    
    model.bigram.forEach((key, value) -> { // for each entry...
    
        try { // has to be used to prevent compiler complaining
        
            writer.write(key + ", " + value + "\n"); // csv format, key is the context, value is the vector
            
        } catch (IOException e) {} // has to be used to prevent compiler complaining
        
    });
    
    writer.close(); // didn't have to do this, still did to be nice
    
} catch (IOException e) {} // has to be used to prevent compiler complaining

This (non Java) code would be shown below. There will be four datasets, words (based on MIT's 10000 words list), boys (based on 1000 boy names in 2025), girls (based on 1000 girl names in 2025), names (based on the 2000 aforementioned boy and girl names). They will each have a unigram version, bigram version, and a trigram version, as indicated by their name.

If you want to test whether the outputs of these are correct, go to the corresponding word file and (for example) search for "ba" in page. If there are, say, 208 cases, this means that the bigram csv file in row "b", column "a" should also have 208 as its value. Searching "sta" in word file means you should look for row "st" and column "a" in the trigram file, etc. For unigram, just search how many x letters there are, and see the columns.

In [2]:
import doctest
import pandas as pd
import numpy as np
import altair as alt
!pip install vega_datasets

words_uni = pd.read_csv("csvs/csv_words_unigram.csv", index_col=0)
words_bi = pd.read_csv("csvs/csv_words_bigram.csv", index_col=0)
words_tri = pd.read_csv("csvs/csv_words_trigram.csv", index_col=0)
boys_uni = pd.read_csv("csvs/csv_boys_unigram.csv", index_col=0)
boys_bi = pd.read_csv("csvs/csv_boys_bigram.csv", index_col=0)
boys_tri = pd.read_csv("csvs/csv_boys_trigram.csv", index_col=0)
girls_uni = pd.read_csv("csvs/csv_girls_unigram.csv", index_col=0)
girls_bi = pd.read_csv("csvs/csv_girls_bigram.csv", index_col=0)
girls_tri = pd.read_csv("csvs/csv_girls_trigram.csv", index_col=0)
names_uni = pd.read_csv("csvs/csv_names_unigram.csv", index_col=0)
names_bi = pd.read_csv("csvs/csv_names_bigram.csv", index_col=0)
names_tri = pd.read_csv("csvs/csv_names_trigram.csv", index_col=0)

# these data are here just as a way to show my work, but they also work 
# well here as inputs to plots. In the original code, these are intermediate steps.
# the above data have not been smoothed nor normalized. The following will be, and 
# it only includes the dataframes I would use for analysis, and is what my code actually uses.
# _sn stands for the smoothed and normalized version of the above data.


# Only these four will be used for perplexity calculation.
words_tri_sn = pd.read_csv("csvs/csv_words_trigram_sn.csv", index_col=0)
boys_tri_sn = pd.read_csv("csvs/csv_boys_trigram_sn.csv", index_col=0)
girls_tri_sn = pd.read_csv("csvs/csv_girls_trigram_sn.csv", index_col=0)
names_tri_sn = pd.read_csv("csvs/csv_names_trigram_sn.csv", index_col=0)

words_bi_sn = pd.read_csv("csvs/csv_words_bigram_sn.csv", index_col=0)
boys_bi_sn = pd.read_csv("csvs/csv_boys_bigram_sn.csv", index_col=0)
girls_bi_sn = pd.read_csv("csvs/csv_girls_bigram_sn.csv", index_col=0)
names_bi_sn = pd.read_csv("csvs/csv_names_bigram_sn.csv", index_col=0)

# these ascii character are used due to the many ascii manipulations in the java code.
# their purposes are explained above in step 2 of data manipulation
nul = "␀"
stx = "␂"
etx = "␃"

words_bi


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,␃
␂,720.0,536.0,1015.0,560.0,466.0,432.0,288.0,334.0,386.0,127.0,...,581.0,1005.0,549.0,127.0,190.0,288.0,12.0,42.0,22.0,0.0
a,6.0,132.0,323.0,237.0,8.0,40.0,147.0,21.0,218.0,3.0,...,678.0,311.0,781.0,83.0,79.0,34.0,17.0,101.0,20.0,299.0
b,160.0,15.0,8.0,4.0,170.0,0.0,1.0,2.0,128.0,8.0,...,116.0,51.0,7.0,101.0,2.0,3.0,0.0,15.0,1.0,53.0
c,362.0,2.0,63.0,9.0,388.0,2.0,2.0,314.0,192.0,1.0,...,131.0,41.0,303.0,120.0,2.0,1.0,0.0,35.0,2.0,169.0
d,148.0,6.0,9.0,36.0,489.0,4.0,26.0,5.0,362.0,9.0,...,71.0,117.0,9.0,85.0,28.0,11.0,1.0,34.0,0.0,880.0
e,377.0,48.0,322.0,685.0,176.0,94.0,94.0,18.0,44.0,3.0,...,1114.0,941.0,276.0,28.0,120.0,73.0,175.0,45.0,4.0,1310.0
f,101.0,1.0,4.0,2.0,121.0,76.0,3.0,1.0,192.0,0.0,...,61.0,7.0,30.0,62.0,0.0,2.0,1.0,11.0,0.0,63.0
g,124.0,6.0,3.0,3.0,274.0,2.0,23.0,104.0,103.0,0.0,...,133.0,62.0,15.0,67.0,0.0,0.0,0.0,24.0,1.0,613.0
h,248.0,4.0,3.0,6.0,294.0,0.0,0.0,1.0,188.0,0.0,...,44.0,14.0,69.0,54.0,1.0,5.0,0.0,30.0,3.0,186.0
i,258.0,87.0,456.0,174.0,282.0,93.0,158.0,1.0,10.0,6.0,...,170.0,441.0,455.0,19.0,184.0,1.0,22.0,1.0,50.0,92.0


## Results

First, to answer the first research question: ***"Are there patterns in how English words are constructed? If so, can the patterns be used to replicate the acceptability judgements of a native English Speaker?"***
The answer would be an obvious yes. There are cases where one letter is more likely to follow the other than anything else. Seeing the probability distribution given a context (which, for unigrams, is no context; for bigrams, is one character preceding the content; for trigrams, is two characters preceding the content.)
(Note: this works for all low-order n-grams, but bigram works the best visually. unigram would return a single bar (because it can appear in any context), while trigram would return too many.)

The fact that it isn't a uniform distribution shows that there are patterns more likely or less likely in English. And yes, I believe the results are self explanatory and intuitive. If some odd cases seems to have a high probability, that would be a problem with the word set (It infamously includes "words" like "xhtml", "xxx", "acdbentity").

**(It is important to know what the bars actually represents here, since there are two different context for it! When the table isn't smoothed or normalized, the column vector that the bar represents simply adds up to all occurences of *content*. This does not mean that when the table is smoothed, the column vector adds up to one! This is because the normalization is done based on row vectors instead of columns. This is best illustrated by seeing the normalized and not normalized words_bi, given content 'u'. 'u' has many potential context, 'o', 's', and 't', but when the row vectors are normalized, 'u' would not stand out from 'o', 's', and 't'. Instead, 'u' would stand out from 'q', because 'q' is usually comes before 'u' in most cases. This shows that normalization shows the "true" context-content pairs.)**

In [3]:
def peek_context(table, content):
    """
    Given an n-gram table and the content input, return a bar chart that shows the likelihood or 
    instances in training set (depending on whether the input is normalized or not) of each context
    (26 characters and start of word). The bar chart would be vertical and sorted in descending.

    Tests: It seems like there are no ways to extract data from altair, according to ChatGPT. The
    chart's top right's option does have a few choices where one can see the full data set this is
    based on, but it doesn't seem like there can be one with just data from the chart.
    I have added tooltips that would appear if you hover over a bar, displaying the bar's value.
    The bar's value can be checked with the charts that underlies them, and if the underlying data
    is not smoothed, it is also possible to go back to the training set to see for yourself.
    (example, if you use peek_context(words_bi, 'u') and see index 'o', content 'u', value 275,
    this would mean that there are 275 cases of "ou" in the MIT word set. This can be tested by
    using the find in page function in Jupyter Notebook)

    There is, however, an edge case with searches like these. "aaa" would count as only 1 case for
    search "aa", but of course, it counts as 2 cases while processing (The first and second 'a', 
    then the second and third.) This is rare, but if you see it, it isn't a mistake with the data.

    I understand that implementation details should not be included in docstring, but I feel like
    something should be here in place of the tests.
    """
    x_title = "Distribution of contexts given \'" + content + "\'"
    y_title = "Given \'" + content + "\' as content"
    return alt.Chart(table.reset_index()).mark_bar().encode(
        alt.X('index', sort='-y', title=x_title),
        alt.Y(content, type="quantitative", title=y_title),
        tooltip=['index', content]
    )

peek_context(words_bi, 'u')

alt.Chart(...)

Aside from returning the column vector, returning the row vector would also be useful. Since the normalization, calculations and generations all use the row vector, this similar method would be just as if not more useful than the column vector, but they all prove one thing: that there are patterns in English.

(Here, a bit more code is needed, because the context varies based on the table input, and how altair deals with plotting. etx has to be ignored due to python not accepting that character.)

In [4]:
def peek_content(table, context):
    """
    Given an n-gram table and the context input, return a bar chart that shows the likelihood or 
    instances in training set (depending on whether the input is normalized or not) of each content
    (26 characters and end of word). The bar chart would be vertical and sorted in descending.

    Tests: please refer to the tests docstring of peek_context. Here, things can be tested in a
    similar way: if there are 137 cases of content 'a' that comes after context "tr", this means
    that there are 137 cases of "tra" if you do a search in the MIT.txt file. Again, this only
    works with the non normalized versions.
    """
    valid = check_input_validity(check_table_gram(table), context)
    if (not valid): 
        raise ValueError("Invalid Input")
    table = table.reset_index()
    row = table.loc[table["index"] == context, slice('a', 'z')].squeeze()
    row = row.reset_index()
    row.columns = ['index', context]

    x_title = "Distribution of contents given \'" + context + "\'"
    y_title = "Given \'" + context + "\' as context"
    return alt.Chart(row).mark_bar().encode(
        alt.X('index', sort='-y', title=x_title),
        alt.Y(context, type="quantitative", title=y_title),
        tooltip=['index', context]
    )

def check_table_gram(table):
    """
    Given an n-gram table, return the "n" of the n-gram.
    >>> check_table_gram(words_uni)
    1
    >>> check_table_gram(words_bi)
    2
    >>> check_table_gram(names_tri)
    3
    """
    if (len(table) == 1):
        return 1
    elif (len(table) <= 30):
        return 2
    else:
        return 3

def check_input_validity(table_gram, context):
    """
    Given the n of an n-gram, and the context given, returns a boolean representing
    whether the context input is valid.
    >>> check_input_validity(1, "*")
    True
    >>> check_input_validity(1, "euih")
    False
    >>> check_input_validity(2, "e")
    True
    """
    if (context == "*"):
        return table_gram == 1
    else:
        return len(context) + 1 == table_gram

doctest.run_docstring_examples(check_table_gram, globals())
doctest.run_docstring_examples(check_input_validity, globals())
peek_content(words_tri, 'tr')

alt.Chart(...)

And the following are a few interesting results:

In [5]:
peek_context(words_bi, 'x')

alt.Chart(...)

In [6]:
peek_content(words_bi, 'q')

alt.Chart(...)

In [7]:
peek_content(words_tri, 'qu')

alt.Chart(...)

Second, let's tackle the second research question: ***"Are there certain combinations of characters that give a word or a name certain impression?"*** For this, we can use heat maps to see if the probability distributions of boy and girl names are any different.

Note that this code IS built to handle trigrams, but due to altair can only handle only 5000 rows of them (MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000)), only very small trigrams with just 185 rows (185*27=4995) can work, which is rare, since the training set with all 118 elements already have 222 rows or 221 trigrams. I tried what they suggested, but it didn't seem to work with Jupyter Notebook. Therefore, only unigrams and trigrams can work here.

Please also note that the same issue with the false normalization would affect the _sn versions of the bigrams. Ignoring the etx content, the heatmap would still be consistent.

In [8]:
def get_heatmap(table):
    """
    Given an n-gram table, return a heatmap with the 27 contexts as the rows and 27 contents as
    columns, and the likelihood/number of appearances as the color value.
    
    Tests: please refer to the tests docstring in peek_context. Here, things can be tested in a 
    similar way, moving your mouse onto the squares would give you a tool tip of the contexts and
    contents and the number of appearances of these pairs. They can be checked in the same way as
    described in peek_context and peek_content.
    """
    table = table.reset_index().melt(id_vars='index', var_name='Contents', value_name='value')
    table = table.rename(columns={'index': 'Contexts'})
    return alt.Chart(table).mark_rect().encode(
        alt.X('Contents:O'),
        alt.Y('Contexts:O'),
        color='value:Q',
        tooltip=['Contexts', 'Contents', 'value']
    )
get_heatmap(words_bi)

alt.Chart(...)

So with this tool, we can see a side by side comparison of the different heatmaps created by boy names and girl names:

In [9]:
get_heatmap(boys_bi) | get_heatmap(girls_bi)

alt.HConcatChart(...)

In [10]:
get_heatmap(boys_uni) | get_heatmap(girls_uni)

alt.HConcatChart(...)

And even though they all follow large trends of English, there are some clear differences, even visible in unigram! Girl names perfer 'a', and ending on 'a', (e.g. Anna, Emma, Olivia, etc.) and the pattern of 'l' being followed by a vowel is more prominent than that of boy names; while boy names has less biases overall except that it likes ending on 'n' (e.g. Aaron, Bjorn, John), and a bunch of "on"s, likely due to the "Jackson"s and "Harrison"s. If the trigram heatmaps were available, it would be used to find more patterns, but alas.

Lastly, ***"Can low-order n-gram generative models capture English phonotactics, simply by the surface analysis of the characters instead of their underlying sounds? If so, how low can n go until the model fails?"*** The second question will be quite clear once we answer the first.

Experimentally (on the Java code that can't be run here), unigram and bigram model were disastrous when it came to generating English sounding words:
(These are generated by the commands "gen words 1 10" and "gen words 2 10")

Ten words generated by unigram: (amnaoia, lmpetyipentnr, obeutdv, osneniren, snlloerf, dswrocua, isaonssbeo, areg, loaetszg, lsiosen)

Ten words generated by bigram: (lejoviorialeq, dinchal, cealdied, erocor, tivenar, sileroco, bhabicocl, matoms, ctiran, shiesevesav)

Trigrams was a bit better:

Ten words generated by trigram: (tompire, earchicin, falfs, plecha, asoroction, mitorme, bdishourga, pasip, bentionsies, naterce)

The unigram was expectedly bad, but bigrams unexpectedly underperformed. This was likely due to the amount of two letter consonants combinations and their relative distributions. For example, "ct", which has many pairs, often appears in the middle or end of words (lactose, abstract, etc.). However, since 'c' is a very common first letter, it is not uncommon for that 'c' to be followed by a 't', leading to 'ct' in the beginning . With only one character input as context, there simply isn't enough memory for it to make the right pairs at the right time. This began to improve with trigrams, but there are still digraphs (like sh, ch, two letters that represent one sound) where it takes up too much context memory for it to perform well enough (reference "earchicin" in the trigram generations).

One solution is to move on from surface (letter) based analysis, and turn to IPA, which represents digraphs with one symbol, and evades some other problem like the inconsistent spelling of English. However, it is likely going to be bloated, and how its many diacritics would be handled can also be affecting the model's performance based on different languages. There are goods of it too, in that currently, the model can only handle languages or latinization that uses the 26 latin alphabet. Switching to IPA would cover almost all languages, but what it can't do is to generate words that also *look* like English.

So, yes, low-order n-gram generative models does capture English phonotactics, and the lowest we can go is three.

=================================================================================================

But that's not qualitative. So let's see some code! Though we have to cover what perplexity is first.

Perplexity is how "perplexed" (or how unlikely it would be something that the model would spit out itself) the model is after seeing your text input. If you have a model that captures the patterns of English, then the higher the perplexity is, the more "un-English" a word is. Perplexity is calculated in many ways, but mainly euler's number raised to the power of the logged sum of all probability of the context-content pairs over negative number of pairs.

In [11]:
def perplexity_bar(word):
    """
    This model takes a string input that is the target of the perplexity calculations, and returns
    the perplexity based on four smoothed and normalized trigram tables, in a vertical bar graph 
    ranked in ascending order. (So that the leftmost one would be what the input most likely is.)

    Tests: Unfortunately, this one is especially hard to test, due to the difficulty of the 
    perplexity formula. Obvious tests would be to see if the girls model does give the lowest
    perplexity when given a girl's name, but the four model's performance are unstable at best.
    My favorite test is "Voila" vs "Viola". It will be shown below!
    """
    data = pd.DataFrame({
        "models": ["words", "boys", "girls", "names"],
        "perplexities": [generate_perplexity(words_tri_sn, word, words_bi_sn), 
                         generate_perplexity(boys_tri_sn, word, boys_bi_sn), 
                         generate_perplexity(girls_tri_sn, word, girls_bi_sn), 
                         generate_perplexity(names_tri_sn, word, names_bi_sn)]
    })
    return alt.Chart(data).mark_bar().encode(
        alt.X('models', sort='y'),
        alt.Y("perplexities", type="quantitative"),
        tooltip=['models', "perplexities"]
    )

def generate_perplexity(table, word, backoff):
    """
    Given a table (assumed to be a smoothed and normalized trigram), a string input, a backoff
    model (used in case of the case not existing in the trigram, expected to be a smoothed and
    normalized bigram), and returns the perplexity for the given trigram model. 

    >>> generate_perplexity(words_tri_sn, "crab", words_bi_sn)
    15.839211131027703
    >>> generate_perplexity(boys_tri_sn, "crab", boys_bi_sn)
    146.9456734867367
    >>> generate_perplexity(girls_tri_sn, "crab", girls_bi_sn)
    59.92717792393581
    >>> generate_perplexity(names_tri_sn, "crab", names_bi_sn)
    63.3391704989425
    """
    temp = "␀␂"
    sum = np.log(table.loc[temp, word[0]])
    for i in range(0, len(word) - 1):
        temp = temp[1] + word[i]
        if temp in table.index:
            sum += np.log(table.loc[temp, word[i + 1]])
        else:
            sum += np.log(backoff.loc[temp[1], word[i + 1]])
    return np.exp(-1 * sum / len(word))

doctest.run_docstring_examples(generate_perplexity, globals())
perplexity_bar("jungle")
# this shows that "jungle" is very likely to be a word, while unlikely to be a name.

alt.Chart(...)

The following comparative charts display the power of this model. Voila and viola, made of the same characters, composed in different ways, are cleverly identified as a word and girl name respectively. I would say that this is a perfect showcase of the model's powers.

In [12]:
perplexity_bar("voila") | perplexity_bar("viola")

alt.HConcatChart(...)

(For fun, I found the word with the least perplexity for each of the models. They are:
The most English English word: constant
The most boyish boy name: Alentin
The most girly girl name: Aliana
The most name-y name: Aleyah (which is more girly than boyish, interestingly enough) )

## Implications and Limitations

I don't believe that this model can be used to hurt anyone (since names only have as much power as language have, and this model likely won't have any effect on languages), and the source of the data are publicly available and are acquired in ethical ways. 

However, if someone believes in these models too much, they can probably use this to dictate what is an English word or not, leading to the classification of words, and people assigning goodness and badness to how English something is. It wouldn't be good if the model were a perfect model of English, especially when it isn't. Words that include rare combination of letters but are still of the English lexicon (like hokkien, hokkaido) have high perplexity ratings compared to other words with latin or greek origin.

Aside from that, since it is not built to handle not English words or names, it can't handle what isn't English. For example, here is my own last name fed into the perplexity barplot method:

In [13]:
perplexity_bar("zhou")

alt.Chart(...)

It was "right" in that it is a boy name (even though there are no boy or girl *last names*), but a 59.25 perplexity being the smallest amongst all four model shouldn't be a surprice due to the odd zh combination.

Lastly, there might be some issue in saying that a name is a certain way, especially when the model doesn't have the most up to date names.

In [14]:
perplexity_bar("jesse") | perplexity_bar("william")

alt.HConcatChart(...)

The models say that "Jesse" is definitely more boyish than girlish, and that "William" is boyish but also fairly girly. I believe that most people would give the opposite judgment, in that Jesse is more ambiguous while William is definitely more masculine. The model isn't an accurate representation of the judgments reality.

In [15]:
# for fun, Jesse vs Jessie is compared here:
perplexity_bar("jesse") | perplexity_bar("jessie")

alt.HConcatChart(...)